###### Table of Contents
* Introduction
* cleaning

###### Introduction
**Dataset Description**
<br>i will go through candy data 2017 , this data has :

* Internal_ID
* Q1-Going Out?: Are you actually going trick or treating yourself? "has values Yes or No"
* Q2-Gender: It has four different options, Femal ,Male ,Other ,I'd rather not say
* Q3-Age: Numerical field
* Q4-Country: Text Field, but users have written their own version of the names. Example, for America, there are entries such as USA, us, US, America so we should consider that while cleanning
* Q5-State/Province: Text Field, but users have written their own version of the names. Same as the country data.
* Q6-Joy Or Despair: All kinds of chocolate bars are the questions with three distinct options to choose from (Joy, Meh, Despair).
* Q7-Joy Other: Text Field to write items not included above that give you JOY. Lots of missing values.
* Q8-Despair Other: Text Field ti write items not included above that give you DESPAIR. Lots of missing values.
* Q9-Other Comments: Text Field. Lots of missing values.
* Q10-Dress: Binary field. Missing values present
* Unnamed: 113
* Q11-Day: Binary Answer Field. Missing values present

###### import packages and read our data

In [1]:
import pandas as pd
import numpy as np
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [2]:
candy=pd.read_csv('candyhierarchy2017.xlsx - responses (2) (1).csv.csv')
candy.head()

,Internal ID,Q1: GOING OUT?,Q2: GENDER,Q3: AGE,Q4: COUNTRY,"Q5: STATE, PROVINCE, COUNTY, ETC",Q6 | 100 Grand Bar,Q6 | Anonymous brown globs that come in black and orange wrappers\t(a.k.a. Mary Janes),Q6 | Any full-sized candy bar,Q6 | Black Jacks,...,Q8: DESPAIR OTHER,Q9: OTHER COMMENTS,Q10: DRESS,Unnamed: 113,Q11: DAY,Q12: MEDIA [Daily Dish],Q12: MEDIA [Science],Q12: MEDIA [ESPN],Q12: MEDIA [Yahoo],"Click Coordinates (x, y)"
0,90258773,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,90272821,No,Male,44,USA,NM,MEH,DESPAIR,JOY,MEH,...,NaN,Bottom line is Twix is really the only candy w...,White and gold,NaN,Sunday,NaN,1.0,NaN,NaN,"(84, 25)"
2,90272829,NaN,Male,49,USA,Virginia,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,90272840,No,Male,40,us,or,MEH,DESPAIR,JOY,MEH,...,NaN,Raisins can go to hell,White and gold,NaN,Sunday,NaN,1.0,NaN,NaN,"(75, 23)"
4,90272841,No,Male,23,usa,exton pa,JOY,DESPAIR,JOY,DESPAIR,...,NaN,NaN,White and gold,NaN,Friday,NaN,1.0,NaN,NaN,"(70, 10)"


In [3]:
candy.shape

(2460, 120)

###### summary

* Missing values are in the standard format (NAN)
* All the columns should be categorial except for the "Q3: AGE" column
* The NANs in the "Q12" columns means "Not selected", should be replaced with 0
* there are unneeded columns for my analysis ("Internal ID", "Unnamed: 113", "Click Coordinates (x, y)"...)
* columns should be renamed for easier access
* There is inconsistent data in the "Q4: COUNTRY" column


### cleaning

###### filling the "Q12" columns NANs with 0

In [4]:
# 0>>>"Not selected", 1>>> "Selected"
cols=["Q12: MEDIA [Daily Dish]","Q12: MEDIA [Science]", "Q12: MEDIA [ESPN]", "Q12: MEDIA [Yahoo]"]
candy[cols]=candy[cols].fillna(0)

**gving variables the correct data types**
* "Q4: COUNTRY", "Q5: STATE, PROVINCE, COUNTY, ETC" colmns are easier to be cleaned as string



In [5]:
df=candy.drop(["Q4: COUNTRY","Q5: STATE, PROVINCE, COUNTY, ETC","Q3: AGE"],axis=1)
candy[df.columns]=candy[df.columns].astype('category')

In [6]:
candy[ "Q3: AGE"]=pd.to_numeric(candy[ "Q3: AGE"], errors='coerce')

In [7]:
candy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2460 entries, 0 to 2459
Columns: 120 entries, Internal ID to Click Coordinates (x, y)
dtypes: category(117), float64(1), object(2)
memory usage: 560.5+ KB


###### droping the useless columns 
* columns I wouldn't use
* columns with too much NANs (more than 80%)

In [8]:
candy.drop(columns=['Internal ID','Unnamed: 113','Click Coordinates (x, y)'],inplace=True)

In [9]:
for column in candy.columns:
    if candy[column].isna().sum()>1968:
        candy.drop([column],axis=1,inplace=True)

In [10]:
candy.shape

(2460, 116)

###### renaming the columns for easiers acces

In [11]:
candy.rename(columns={'Q1: GOING OUT?':'going out','Q2: GENDER':'gender', 'Q3: AGE':'age', 'Q4: COUNTRY':'country',
                      
                      'Q5: STATE, PROVINCE, COUNTY, ETC':'area','Q7: JOY OTHER':'joy_other', 'Q8: DESPAIR OTHER':'despair_othr',
                      
        'Q9: OTHER COMMENTS':'other_comments','Q10: DRESS':'dress', 'Q11: DAY':'day','Q12: MEDIA [Daily Dish]':'media_DailyDish'
                      ,
       'Q12: MEDIA [Science]':'media_science', 'Q12: MEDIA [ESPN]':'media_ESPN', 'Q12: MEDIA [Yahoo]':'media_yahoo',}
             ,inplace=True)

In [13]:
candy.columns

Index(['going out', 'gender', 'age', 'country', 'area', 'Q6 | 100 Grand Bar',
       'Q6 | Anonymous brown globs that come in black and orange wrappers\t(a.k.a. Mary Janes)',
       'Q6 | Any full-sized candy bar', 'Q6 | Black Jacks',
       'Q6 | Bonkers (the candy)',
       ...
       'Q6 | Whole Wheat anything', 'Q6 | York Peppermint Patties',
       'joy_other', 'despair_othr', 'dress', 'day', 'media_DailyDish',
       'media_science', 'media_ESPN', 'media_yahoo'],
      dtype='object', length=116)

###### removing the empty rows

In [14]:
candy.dropna(axis=0, how='all',inplace=True)

In [15]:
candy.shape

(2460, 116)

###### "going out" column

In [16]:
candy['going out'].value_counts(dropna=False)

No     2038
Yes     312
NaN     110
Name: going out, dtype: int64

In [17]:
# fill  NANs with mode
candy['going out'].fillna(candy['going out'].mode()[0],inplace=True)

###### "gender" column

In [18]:
candy['gender'].unique()

[NaN, 'Male', 'Female', 'I'd rather not say', 'Other']
Categories (4, object): ['Female', 'I'd rather not say', 'Male', 'Other']

In [19]:
candy['gender'].fillna("I'd rather not say",inplace=True)

###### "age" column

In [20]:
# data got ouliers
candy['age'].describe()

count    2351.000000
mean       42.605444
std        23.844665
min         1.000000
25%        34.000000
50%        41.000000
75%        50.000000
max      1000.000000
Name: age, dtype: float64

In [21]:
# filling NANs with the median
candy['age'].fillna(candy['age'].median(),inplace=True)

In [22]:
# handeling Outliers
Q1, Q3= np.percentile(candy['age'],[25,75])
IQR= Q3 - Q1
max= Q3 + (1.5 * IQR)
min = Q1 - (1.5 * IQR)
candy["age"] = np.where(candy["age"]< min, min,candy['age'])
candy["age"] = np.where(candy["age"]>max , max,candy['age'])

In [23]:
candy['age'].describe()

count    2460.000000
mean       42.001382
std        11.541018
min        11.500000
25%        34.000000
50%        41.000000
75%        49.000000
max        71.500000
Name: age, dtype: float64

###### "country" column

In [24]:
candy['country'].unique()

array([nan, 'USA ', 'USA', 'us', 'usa', 'canada', 'Canada', 'Us', 'US',
       'Murica', 'United States', 'uk', 'United Kingdom', 'united states',
       'Usa', 'United States ', 'United staes',
       'United States of America', 'UAE', 'England', 'UK', 'canada ',
       'Mexico', 'United states', 'u.s.a.', 'USAUSAUSA', 'america', '35',
       'france', 'United States of America ', 'U.S.A.', 'finland',
       'unhinged states', 'Canada ', 'united states of america',
       'US of A', 'Unites States', 'The United States', 'North Carolina ',
       'Unied States', 'Netherlands', 'germany', 'Europe', 'Earth', 'U S',
       'u.s.', 'U.K. ', 'Costa Rica', 'The United States of America',
       'unite states', 'U.S.', '46', 'cascadia', 'Australia',
       'insanity lately', 'Greece', 'USA? Hard to tell anymore..',
       "'merica", 'usas', 'Pittsburgh', '45', 'United State', '32',
       'France', 'australia', 'A', 'Can', 'Canae', 'New York',
       'Trumpistan', 'Ireland', 'United Sates', '

In [25]:
USA=[x for x in candy['country'].unique() if (fuzz.WRatio(str(x),'USA')>=70)
     or (fuzz.partial_ratio(str(x),'United States of America')>=70)
     or (fuzz.WRatio(str(x),'U.S.')>=70)]
candy['country'].replace(USA,'USA',inplace=True)

In [26]:
Canada=[x for x in candy['country'].unique() if (fuzz.WRatio(str(x),'Canada')>=70)]
candy['country'].replace(Canada,'Canada',inplace=True)

In [27]:
Uk =[x for x in candy['country'].unique() if (fuzz.partial_ratio(str(x),'United Kingdom')>=60)
     or (fuzz.WRatio(str(x),'UK')>=60)]
candy['country'].replace(Uk,'Uk',inplace=True)
Uk

['uk', 'United Kingdom', 'UK', 'U.K. ', 'Uk', 'UK ', 'United kingdom']

In [28]:
Germany=[x for x in candy['country'].unique() if (fuzz.WRatio(str(x),'Germany')>=70)]
candy['country'].replace(Germany,'Germany',inplace=True)
Germany

['germany', 'Germany']

In [29]:
candy['country'].value_counts(dropna=False).head(10)

USA            2038
Canada          228
NaN              64
Uk               33
Germany          10
Netherlands       6
Australia         5
Japan             5
Scotland          4
Mexico            4
Name: country, dtype: int64

In [30]:
# creating others category that holds all values occurred less tha 5 times
others=[x for x in candy['country'] if candy['country'].value_counts(dropna=False)[x]<5]
candy['country'].replace(others,'others',inplace=True)

In [31]:
# fill NANs with others
candy['country'].fillna('others',inplace=True)

In [32]:
candy['country'].value_counts()

USA            2038
Canada          228
others          135
Uk               33
Germany          10
Netherlands       6
Australia         5
Japan             5
Name: country, dtype: int64

**now we change the data type to category**

In [33]:
candy["country"]=candy["country"].astype('category')

###### "Q6" columns

In [39]:
types=[x for x in candy.columns if "Q6" in x]

In [40]:
candy[types]=candy[types].fillna(candy.mode().iloc[0])

###### "dress" column

In [41]:
candy['dress'].value_counts(dropna=False)

White and gold    1081
NaN                744
Blue and black     635
Name: dress, dtype: int64

In [42]:
# creat new category called unkown
candy["dress"] = candy["dress"].cat.add_categories("Unknown")

In [43]:
# fill NANs with Unknown
candy.dress.fillna("Unknown",inplace=True)

###### "day" column

In [44]:
candy['day'].value_counts(dropna=False)

Friday    1091
NaN        725
Sunday     644
Name: day, dtype: int64

In [45]:
# creat new category called unkown
candy["day"] = candy["day"].cat.add_categories("Unknown")

In [46]:
# fill NANs with Unknown
candy.day.fillna("Unknown",inplace=True)

###### finding out the 10 most loved and most hated chocolates.

In [57]:
# very low amount, so we wouldn't take it in count
candy["joy_other"].value_counts().head(10)

Almond Joy            10
Beer                  10
beer                   3
Mounds, Almond Joy     3
Kittens                3
Sex                    3
Cans of pop            2
Charleston Chew        2
White chocolate        2
Whoppers               2
Name: joy_other, dtype: int64

In [58]:
# very low amount, so we wouldn't take it in count
candy['despair_othr'].value_counts().head(10)

Trump                 7
Almond Joy, Mounds    3
Popcorn balls         3
Tootsie Rolls         3
Pennies               3
Donald Trump          2
popcorn               2
Red hots              2
Apples                2
Restaurant mints      2
Name: despair_othr, dtype: int64

In [47]:
classified=candy[types].apply(pd.Series.value_counts)
classified

,Q6 | 100 Grand Bar,Q6 | Anonymous brown globs that come in black and orange wrappers\t(a.k.a. Mary Janes),Q6 | Any full-sized candy bar,Q6 | Black Jacks,Q6 | Bonkers (the candy),Q6 | Bonkers (the board game),Q6 | Bottle Caps,Q6 | Box'o'Raisins,Q6 | Broken glow stick,Q6 | Butterfinger,...,Q6 | Three Musketeers,Q6 | Tolberone something or other,Q6 | Trail Mix,Q6 | Twix,"Q6 | Vials of pure high fructose corn syrup, for main-lining into your vein",Q6 | Vicodin,Q6 | Whatchamacallit Bars,Q6 | White Bread,Q6 | Whole Wheat anything,Q6 | York Peppermint Patties
DESPAIR,85,1823,17,1751,495,546,560,1867,2331,141,...,176,84,1544,71,1938,1512,288,2212,2036,232
JOY,1620,176,2231,92,116,193,465,117,24,1859,...,1766,2026,228,2103,233,707,1663,44,117,1810
MEH,755,461,212,617,1849,1721,1435,476,105,460,...,518,350,688,286,289,241,509,204,307,418


In [53]:
# ordering with Joy to get top 10 most loved
classified.sort_values(by="JOY",axis=1,ascending=False)

,Q6 | Any full-sized candy bar,Q6 | Reese’s Peanut Butter Cups,Q6 | Kit Kat,"Q6 | Cash, or other forms of legal tender",Q6 | Twix,Q6 | Snickers,Q6 | Tolberone something or other,Q6 | Lindt Truffle,Q6 | Peanut M&M’s,Q6 | Nestle Crunch,...,Q6 | Kale smoothie,Q6 | Jolly Rancher (bad flavor),Q6 | Black Jacks,Q6 | Real Housewives of Orange County Season 9 Blue-Ray,Q6 | Dental paraphenalia,Q6 | JoyJoy (Mit Iodine!),Q6 | White Bread,Q6 | Gum from baseball cards,Q6 | Candy that is clearly just the stuff given out for free at restaurants,Q6 | Broken glow stick
DESPAIR,17,95,52,65,71,79,84,98,121,102,...,2179,1755,1751,2233,2125,1982,2212,2178,2016,2331
JOY,2231,2177,2121,2119,2103,2089,2026,1995,1962,1886,...,108,98,92,91,91,76,44,44,39,24
MEH,212,188,287,276,286,292,350,367,377,472,...,173,607,617,136,244,402,204,238,405,105


###### Top 10 loved chocolates
* Any full-sized candy bar
* Reese’s Peanut Butter Cups
* Kit Kat
* Cash, or other forms of legal tender
* Twix
* Snickers
* Tolberone something or other
* Lindt Truffle
* Peanut M&M’s
* Nestle Crunch	

In [52]:
# ordering with DESPAIR to get top 10 most hated
classified.sort_values(by="DESPAIR",axis=1,ascending=False)

,Q6 | Broken glow stick,Q6 | Real Housewives of Orange County Season 9 Blue-Ray,Q6 | White Bread,Q6 | Kale smoothie,Q6 | Gum from baseball cards,Q6 | Dental paraphenalia,Q6 | Creepy Religious comics/Chick Tracts,Q6 | Whole Wheat anything,Q6 | Spotted Dick,Q6 | Candy that is clearly just the stuff given out for free at restaurants,...,Q6 | Lindt Truffle,Q6 | Reese’s Peanut Butter Cups,Q6 | 100 Grand Bar,Q6 | Tolberone something or other,Q6 | Snickers,Q6 | Regular M&Ms,Q6 | Twix,"Q6 | Cash, or other forms of legal tender",Q6 | Kit Kat,Q6 | Any full-sized candy bar
DESPAIR,2331,2233,2212,2179,2178,2125,2110,2036,2022,2016,...,98,95,85,84,79,75,71,65,52,17
JOY,24,91,44,108,44,91,246,117,128,39,...,1995,2177,1620,2026,2089,1797,2103,2119,2121,2231
MEH,105,136,204,173,238,244,104,307,310,405,...,367,188,755,350,292,588,286,276,287,212


###### Top 10 most hated
* Broken glow stick
* Real Housewives of Orange County Season 9 Blue-Ray
* White Bread
* Kale smoothie
* Gum from baseball cards
* Dental paraphenalia
* Creepy Religious comics/Chick Tracts
* Whole Wheat anything
* Spotted Dick
* Candy that is clearly just the stuff given out for free at restaurants